In [14]:
import geopandas as gpd
import h3
from shapely.geometry import shape, Polygon, MultiPolygon, box
from concurrent.futures import ThreadPoolExecutor
import numpy as np

In [15]:
# Задаем уровень зума
resolution = 5  # H3 zoom level
num_splits = 4  # Количество разбиений по осям

source_geojson = 'data/extent.geojson'
target_geopackage = f'data/h3_z{resolution}.gpkg'

In [17]:
import geopandas as gpd
import h3
from shapely.geometry import shape, Polygon, MultiPolygon, box
from concurrent.futures import ThreadPoolExecutor
import numpy as np

# Чтение GeoJSON файла
input_geojson_file = source_geojson
geojson_data = gpd.read_file(input_geojson_file)

# Функция для разбиения полигона на части
def split_polygon(polygon, num_splits):
    min_x, min_y, max_x, max_y = polygon.bounds
    x_splits = np.linspace(min_x, max_x, num_splits + 1)
    y_splits = np.linspace(min_y, max_y, num_splits + 1)
    sub_polygons = []
    for i in range(num_splits):
        for j in range(num_splits):
            sub_poly = box(x_splits[i], y_splits[j], x_splits[i+1], y_splits[j+1])
            intersection = polygon.intersection(sub_poly)
            if not intersection.is_empty:
                if isinstance(intersection, (Polygon, MultiPolygon)):
                    sub_polygons.append(intersection)
    return sub_polygons

# Функция для получения H3 ячеек для одного полигона
def get_h3_cells(polygon, resolution):
    cells = h3.polyfill(polygon.__geo_interface__, resolution)
    return cells

# Функция для конвертации H3 ячеек в полигоны
def h3_cells_to_polygons_with_index(h3_cells):
    data = []
    for cell in h3_cells:
        # Получаем координаты вершин полигона (H3 ячейки)
        boundary = h3.h3_to_geo_boundary(cell, geo_json=True)
        # Создаем полигон с учетом порядка координат
        polygon = Polygon([(point[1], point[0]) for point in boundary])
        # Добавляем полигон и соответствующий H3 индекс в список данных
        data.append({'geometry': polygon, 'h3_index': cell})
    return data

# Основная функция для обработки одного под-полигона
def process_sub_polygon(polygon, resolution):
    h3_cells = get_h3_cells(polygon, resolution)
    return h3_cells_to_polygons_with_index(h3_cells)


# Создаем список всех частей полигона
sub_polygons = []
for _, row in geojson_data.iterrows():
    sub_polygons.extend(split_polygon(row['geometry'], num_splits))

# Параллельная обработка частей полигона
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_sub_polygon, poly, resolution) for poly in sub_polygons]
    results = [f.result() for f in futures]

# Объединение всех результатов в один GeoDataFrame
all_data = [item for result in results for item in result]
h3_gdf = gpd.GeoDataFrame(all_data, crs="EPSG:4326")

# Сохранение в файл GeoPackage с добавлением поля 'h3_index'
output_gpkg_file = target_geopackage
h3_gdf.to_file(output_gpkg_file, layer='h3_cells', driver="GPKG")

print(f"Total number of H3 cells: {len(h3_gdf)}")
print(f"H3 ячейки успешно сохранены в {output_gpkg_file}")


Total number of H3 cells: 297
H3 ячейки успешно сохранены в hexagrid/data/h3_z5.gpkg
